In [45]:
import requests
import pandas as pd
import numpy as np
import json
import sqlite3

In [47]:
key_id = 'f2a86c70ab2e4a96bdea'
service_id = 'I2790'
data_type = 'json'
url = 'http://openapi.foodsafetykorea.go.kr/api'

str_idx = 1
end_idx = 1000
records = []

while True:
    query = f'/{key_id}/{service_id}/{data_type}/{str(str_idx)}/{str(end_idx)}'
    request_url = url + query

    try:
        response = requests.get(request_url, timeout=900)  # 15분(600초) 제한 시간 설정
        response.raise_for_status()  # 오류 발생 시 예외 처리

        data = json.loads(response.text)
        total_count = 90608
        records += data['I2790']['row']

        if end_idx >= total_count:
            break

        str_idx += 1000
        end_idx += 1000
        print(str_idx)

    except requests.exceptions.Timeout:
        print('요청 시간이 초과되었습니다. 프로그램을 중단합니다.')
        break
    except requests.exceptions.RequestException as e:
        print('오류가 발생했습니다:', e)
        break

df = pd.DataFrame(records)
print(df.head())


오류가 발생했습니다: HTTPConnectionPool(host='openapi.foodsafetykorea.go.kr', port=80): Read timed out.
Empty DataFrame
Columns: []
Index: []


In [6]:
df

,NUTR_CONT8,NUTR_CONT9,NUTR_CONT4,NUTR_CONT5,NUTR_CONT6,NUM,NUTR_CONT7,NUTR_CONT1,NUTR_CONT2,SUB_REF_NAME,...,RESEARCH_YEAR,MAKER_NAME,GROUP_NAME,SERVING_SIZE,SAMPLING_REGION_NAME,SAMPLING_MONTH_CD,SAMPLING_MONTH_NAME,DESC_KOR,SAMPLING_REGION_CD,FOOD_CD
48995,0,0,0,0.7,0,48996,0,8,0.7,식약처('20),...,2020,㈜팜크로스,"과자류, 빵류 또는 떡류",2,전국(대표),AVG,평균,아르기닌1000,ZZ,P057273
48996,0,0,0.2,2,0,48997,0,40,9,식약처('20),...,2020,㈜팜크로스,"과자류, 빵류 또는 떡류",10,전국(대표),AVG,평균,함소아 아듀물사마,ZZ,P057274
48997,,,0,,0,48998,,0,0.7,식약처('20),...,2020,(주)한국씨엔에스팜,"과자류, 빵류 또는 떡류",1.2,전국(대표),AVG,평균,더모티브 컴플렉스 탭,ZZ,P057275
48998,0,,0,0,0,48999,,1.9,0,식약처('20),...,2020,(주)한국씨엔에스팜,"과자류, 빵류 또는 떡류",0.5,전국(대표),AVG,평균,돌피니 와트,ZZ,P057276
48999,,,0,0,0,49000,,3,0.7,식약처('20),...,2020,(주)한국씨엔에스팜,"과자류, 빵류 또는 떡류",0.8,전국(대표),AVG,평균,통라이프 레드3컷 레몬밤,ZZ,P057277


In [37]:
drop_columns = ['NUM', 'SAMPLING_REGION_NAME', 'SAMPLING_MONTH_CD', 'FOOD_CD', 'SAMPLING_REGION_CD', 'SAMPLING_MONTH_NAME', 'RESEARCH_YEAR', 'SUB_REF_NAME']
food = df.copy()
food.drop(drop_columns, axis = 1, inplace=True)
food = food[['DESC_KOR', 'SERVING_SIZE', 'NUTR_CONT1', 'NUTR_CONT2', 'NUTR_CONT3', 'NUTR_CONT4', 'NUTR_CONT5', 'NUTR_CONT6', 'NUTR_CONT7', 'NUTR_CONT8', 'NUTR_CONT9', 'GROUP_NAME', 'MAKER_NAME']]
food.iloc[:, 2:11].replace('', np.nan, inplace=True)
food.dropna(inplace=True)
food.reset_index(drop=True, inplace=True)
food = food.astype({
    'SERVING_SIZE' : 'float',
    'NUTR_CONT1' : 'float',
    'NUTR_CONT2' : 'float',
    'NUTR_CONT3' : 'float',
    'NUTR_CONT4' : 'float',
    'NUTR_CONT5' : 'float',
    'NUTR_CONT6' : 'float',
    'NUTR_CONT7' : 'float',
    'NUTR_CONT8' : 'float',
    'NUTR_CONT9' : 'float'
})
new_columns = {
    'DESC_KOR' : 'Name',
    'SERVING_SIZE' : 'Serving_size',
    'NUTR_CONT1' : 'Calories',
    'NUTR_CONT2' : 'Total_carbohydrate',
    'NUTR_CONT3' : 'Protein',
    'NUTR_CONT4' : 'Total_fat',
    'NUTR_CONT5' : 'Sugars',
    'NUTR_CONT6' : 'Sodium',
    'NUTR_CONT7' : 'Cholesterol',
    'NUTR_CONT8' : 'Saturated_fat',
    'NUTR_CONT9' : 'Trans_fat',
    'MAKER_NAME' : 'Maker_name',
    'GROUP_NAME' : 'Group_name'
}
food.rename(columns=new_columns, inplace=True)

# 칼로리 계산 규칙에 따라 각 영양소의 칼로리 계산
protein_calories_per_gram = 4  # 단백질 1g당 칼로리
carb_calories_per_gram = 4  # 탄수화물 1g당 칼로리
fat_calories_per_gram = 9  # 지방 1g당 칼로리

# 각 영양소의 칼로리 계산
food['Protein_Calories'] = food['Protein'] * protein_calories_per_gram
food['Carbohydrate_Calories'] = food['Total_carbohydrate'] * carb_calories_per_gram
food['Fat_Calories'] = food['Total_fat'] * fat_calories_per_gram

food['Total_Calories'] = food['Protein_Calories'] + food['Carbohydrate_Calories'] + food['Fat_Calories']

# 이상한 row 삭제
# Threshold 값 설정
threshold = 100

# 'Calories'와 'Total_Calories' 차이가 threshold보다 큰 row 삭제
food = food[abs(food['Calories'] - food['Total_Calories']) <= threshold]
food.reset_index(drop=True, inplace=True)
food

C:\Users\gyte1\AppData\Local\Temp\ipykernel_30784\1516341076.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food.iloc[:, 2:11].replace('', np.nan, inplace=True)


,Name,Serving_size,Calories,Total_carbohydrate,Protein,Total_fat,Sugars,Sodium,Cholesterol,Saturated_fat,Trans_fat,Group_name,Maker_name,Protein_Calories,Carbohydrate_Calories,Fat_Calories,Total_Calories
0,꿩불고기,500.0,368.80,39.70,33.50,8.50,16.90,1264.31,106.18,1.90,0.10,,,134.00,158.8,76.50,369.30
1,닭갈비,400.0,595.61,44.90,45.90,25.80,21.20,1535.83,193.40,6.00,0.20,,,183.60,179.6,232.20,595.40
2,닭갈비,300.0,558.47,23.10,45.50,31.60,8.50,1016.94,217.67,11.30,0.30,,,182.00,92.4,284.40,558.80
3,닭꼬치,70.0,176.72,13.35,11.56,8.57,3.15,286.91,31.44,2.32,0.04,,,46.24,53.4,77.13,176.77
4,더덕구이,100.0,184.00,31.10,3.10,5.20,11.60,743.37,0.12,0.80,0.00,,,12.40,124.4,46.80,183.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36098,하트쿨 민트캔디,10.0,40.00,10.00,0.00,0.05,0.00,0.00,0.00,0.05,0.00,"과자류, 빵류 또는 떡류",(주)지에스바이오,0.00,40.0,0.45,40.45
36099,하트쿨 자일리톨 비타민캔디,10.0,40.00,10.00,0.00,0.05,0.00,0.00,0.00,0.05,0.00,"과자류, 빵류 또는 떡류",(주)지에스바이오,0.00,40.0,0.45,40.45
36100,한동안,7.2,27.00,4.80,2.40,0.00,4.00,5.00,0.00,0.00,0.00,"과자류, 빵류 또는 떡류",(주)지에스바이오,9.60,19.2,0.00,28.80
36101,아르기닌1000,2.0,8.00,0.70,1.00,0.00,0.70,0.00,0.00,0.00,0.00,"과자류, 빵류 또는 떡류",㈜팜크로스,4.00,2.8,0.00,6.80


In [40]:
food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36103 entries, 0 to 36102
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   36103 non-null  object 
 1   Serving_size           36103 non-null  float64
 2   Calories               36103 non-null  float64
 3   Total_carbohydrate     36103 non-null  float64
 4   Protein                36103 non-null  float64
 5   Total_fat              36103 non-null  float64
 6   Sugars                 36103 non-null  float64
 7   Sodium                 36103 non-null  float64
 8   Cholesterol            36103 non-null  float64
 9   Saturated_fat          36103 non-null  float64
 10  Trans_fat              36103 non-null  float64
 11  Group_name             36103 non-null  object 
 12  Maker_name             36103 non-null  object 
 13  Protein_Calories       36103 non-null  float64
 14  Carbohydrate_Calories  36103 non-null  float64
 15  Fa

In [41]:
food.describe()

,Serving_size,Calories,Total_carbohydrate,Protein,Total_fat,Sugars,Sodium,Cholesterol,Saturated_fat,Trans_fat,Protein_Calories,Carbohydrate_Calories,Fat_Calories,Total_Calories
count,36103.000000,36103.000000,36103.000000,36103.000000,36103.000000,36103.000000,36103.000000,36103.000000,36103.000000,36103.000000,36103.000000,36103.000000,36103.000000,36103.000000
mean,133.175272,231.220418,32.185099,6.124464,8.669079,12.527318,363.954791,17.375015,3.433181,0.073388,24.497857,128.740396,78.021711,231.259964
std,203.993350,614.247064,121.600458,9.759106,24.951600,91.895971,768.685847,48.929147,13.024902,1.648904,39.036425,486.401830,224.564400,614.355831
min,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,65.000000,9.000000,0.800000,0.600000,2.000000,22.340000,0.000000,0.000000,0.000000,3.200000,36.000000,5.400000,64.400000
50%,100.000000,161.000000,22.000000,3.830000,3.700000,7.000000,95.000000,0.000000,1.190000,0.000000,15.320000,88.000000,33.300000,162.500000
75%,180.000000,320.000000,44.875000,8.000000,11.000000,16.000000,400.000000,15.000000,4.200000,0.000000,32.000000,179.500000,99.000000,321.000000
max,22000.000000,86395.000000,20999.000000,600.000000,3466.000000,16200.000000,19524.000000,2220.000000,2141.000000,213.000000,2400.000000,83996.000000,31194.000000,86396.000000


In [42]:
# DataFrame을 SQLite 데이터베이스 파일로 저장
db_path = 'Food.db'
conn = sqlite3.connect(db_path)
food.to_sql('nutrition_facts', conn, index=False, if_exists='replace')
conn.close()

# SQLite 데이터베이스에 연결하여 데이터 확인
conn = sqlite3.connect(db_path)
query = 'SELECT * FROM nutrition_facts'
result = pd.read_sql_query(query, conn)
conn.close()

# 데이터 확인
print(result)

                 Name  Serving_size  Calories  Total_carbohydrate  Protein  \
0                꿩불고기         500.0    368.80               39.70    33.50   
1                 닭갈비         400.0    595.61               44.90    45.90   
2                 닭갈비         300.0    558.47               23.10    45.50   
3                 닭꼬치          70.0    176.72               13.35    11.56   
4                더덕구이         100.0    184.00               31.10     3.10   
...               ...           ...       ...                 ...      ...   
36098        하트쿨 민트캔디          10.0     40.00               10.00     0.00   
36099  하트쿨 자일리톨 비타민캔디          10.0     40.00               10.00     0.00   
36100             한동안           7.2     27.00                4.80     2.40   
36101        아르기닌1000           2.0      8.00                0.70     1.00   
36102       함소아 아듀물사마          10.0     40.00                9.00     0.15   

       Total_fat  Sugars   Sodium  Cholesterol  Saturated_fat  

In [43]:
from sklearn.neighbors import NearestNeighbors
import joblib
import os

# 입력 변수(X)로 데이터 설정
X = food[['Total_carbohydrate', 'Protein', 'Total_fat']]  # 탄수화물, 단백질, 지방 열에 해당하는 컬럼명으로 수정해주세요.

# KNN 모델 생성 및 학습
k = 10  # 최근접 이웃 개수 설정
knn_model = NearestNeighbors(n_neighbors=k)
knn_model.fit(X)

# 저장할 파일 경로
file_path = 'knn_model.pkl'

# 파일이 이미 존재하는지 확인 후 덮어쓰기
if os.path.exists(file_path):
    os.remove(file_path)  # 기존 파일 삭제

# 모델 저장
joblib.dump(knn_model, file_path)

['knn_model.pkl']